In [1]:
import yaml
import pystache
from IPython.display import display, HTML 

def numeric_addr(bitlist):
    out = 0
    for bit in bitlist:
        out = (out << 1) | bit
    return out

def bitfield(n):
    return [int(digit) for digit in bin(n)[2:]]

def resolve(device, incr):
    idx = 0
    remap = {}
    
    if incr > device['adjustable_bits']**2:
        raise Exception("Not enough address pins") 
    
    bitlist = device['address_struct']
    
    if incr > 0:
        adjust = bitfield(incr)
        adjust.reverse()
        
        ## Replace strings with values from a bit representation of the increment
        for value in adjust:
                
            if "A0" in bitlist:
                key = "A{}".format(idx)
                if key not in bitlist:
                    raise Exception("Expecting '{}' in address, not found".format(key))
                remap[key] = value
                idx += 1
                
            else:
                for digit in reversed(bitlist):
                    if not str(digit).isdigit():
                        if digit not in remap.keys():
                            remap[digit] = value
                        else:
                            raise Exception("Not enough address pins") 
                    
        bitlist = [remap.get(n, n) for n in bitlist]

    ## Replace any strings left over with zeros
    return ([0 if isinstance(n, str) else n for n in bitlist], remap)

class Devices:

    def __init__(self, path='./i2c-addresses.yml'):

        self.data = {}
        with open(path) as f:
            self.data = yaml.safe_load(f)

        self.devices = []
        self.taken = {}
    
    def add(self, name):
        if name in self.data:
            addr = self.data[name]
            data = dict(
                name=name, 
                address_struct=addr, 
                adjustable_bits = 7 - addr.count(0) - addr.count(1)
            )
            self.devices.append(data)
        else:
            raise Exception("Cannot find device: {}".format(name))

    def solve(self, html=False, debug=False, stdout=False):
        ## Iterate through the selected devices by the number of adjustable bits
        self.devices = sorted(self.devices, key=lambda d: d['adjustable_bits'])
        
        for idx, dev in enumerate(self.devices):
            incr = 0

            while True:
                try:
                    bits, remap = resolve(dev, incr)
                    address = numeric_addr(bits)                
                except Exception:
                    if debug:
                        print("WARN : Cannot resolve {} in slot {}, no free addresses".format(dev['name'], idx))
                    break

                if address not in self.taken:
                    dev['bits'] = bits
                    dev['address'] = numeric_addr(bits)
                    dev['remap'] = remap
                    self.taken[address] = dev
                    break

                incr += 1
                    
        if stdout:
            self.print()
            
        if html:
            self.print_html()
                

    def print(self):
        for addr, dev in self.taken.items():
            print("{} @ {}".format(dev['name'], hex(addr)))
            print("\t", dev['address_struct'])
            print("\t", dev['bits'])
            print()
            
    def print_table(self):
        out = []
        out.append(["IDX", "Name", "Address", "Configuration"])
        
        for idx, dev in enumerate(self.devices):
            if 'address' in dev:
                addr = hex(dev['address'])
            else:
                addr = "NONE"
            
            out.append([idx, dev['name'], addr])
            
        return out
    
    def print_html(self, raw=False):
        template = u"""<table>
        <tr> <th>IDX</th> <th>Device</th> <th>Address</th> <th>Address Struct</th> <th>Mapping</th> </tr>
        {{#device}}<tr>
            <td>{{idx}}</td> <td>{{name}}</td> <td>{{{address_formatted}}}</td> <td>{{{address_string}}}</td> <td>{{mapping_string}}</td>
        </tr>{{/device}}
        </table>"""
        
        out = []
        
        for idx,dev in enumerate(self.devices):
            
            if 'address' in dev:
                addr = hex(dev['address'])
            else:
                dev['bits'] = []
                addr = "<span style='color:red;'><b>NONE</b></span>"
              
            dev['idx'] = idx
            dev['address_formatted'] = addr
            
            mapping = zip(dev['address_struct'], dev['bits'])
            mapping = ["" if a==b else "{}:{}".format(a,b) for (a,b) in list(mapping)]
            dev['mapping_string'] = "  ".join([str(b) for b in mapping])
            
            address = [str(b) for b in dev['address_struct']]
            address = [b if b.isdigit() else "<b>{}</b>".format(b) for b in address]
            dev['address_string'] = " ".join(address)
            
            out.append(dev)
      
        html = pystache.render(template, dict(device=out))
        
        if raw:
            return html
        else:
            display(HTML(html))

In [4]:
devices = Devices()

devices.add('TLC59116')
devices.add('TLC59116')
devices.add('UCS2113-1')
devices.add('UCS2113-2')
devices.add('MCP23008')
devices.add('24AA025E48')
devices.add('ATECC508A')
devices.add('ATECC508A')
devices.add('MCP9808')

devices.solve(html=True)

IDX,Device,Address,Address Struct,Mapping
0,UCS2113-1,0x57,1 0 1 0 1 1 1,
1,UCS2113-2,0x58,1 0 1 1 0 0 0,
2,ATECC508A,0x60,1 1 0 0 0 0 0,
3,ATECC508A,NONE,1 1 0 0 0 0 0,
4,24AA025E48,0x50,1 0 1 0 0 A1 A0,A1:0 A0:0
5,MCP23008,0x20,0 1 0 0 A2 A1 A0,A2:0 A1:0 A0:0
6,MCP9808,0x18,0 0 1 1 A2 A1 A0,A2:0 A1:0 A0:0
7,TLC59116,0x61,1 1 0 A3 A2 A1 A0,A3:0 A2:0 A1:0 A0:1
8,TLC59116,0x62,1 1 0 A3 A2 A1 A0,A3:0 A2:0 A1:1 A0:0
